most basic model using just the historical ranking data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## load data

In [19]:
# historical match results
matches = pd.read_csv("../data/results.csv") #parse_dates=['date'])

# monthly ranking since Aug 1993
rankings = pd.read_csv("../data/fifa_ranking.csv",) #parse_dates=['rank_date'])

# 2018 match
matches_2018 = pd.read_csv("../data/World Cup 2018 Dataset.csv")
matches.shape, rankings.shape, matches_2018.shape

((39008, 9), (57793, 16), (33, 20))

In [20]:
matches.head()
# neutral: match is in 3rd country

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [21]:
matches_2018.head()

,Team,Group,Previous appearances,Previous titles,Previous finals,Previous semifinals,Current FIFA rank,First match against,Match index,history with first opponent W-L,history with first opponent goals,Second match against,Match index.1,history with second opponent W-L,history with second opponent goals,Third match against,Match index.2,history with third opponent W-L,history with third opponent goals,Unnamed: 19
0,Russia,A,10.0,0.0,0.0,1.0,65.0,Saudi Arabia,1.0,-1.0,-2.0,Egypt,17.0,NaN,NaN,Uruguay,33.0,0.0,0.0,NaN
1,Saudi Arabia,A,4.0,0.0,0.0,0.0,63.0,Russia,1.0,1.0,2.0,Uruguay,18.0,1.0,1.0,Egypt,34.0,-5.0,-5.0,NaN
2,Egypt,A,2.0,0.0,0.0,0.0,31.0,Uruguay,2.0,-1.0,-2.0,Russia,17.0,NaN,NaN,Saudi Arabia,34.0,5.0,5.0,NaN
3,Uruguay,A,12.0,2.0,2.0,5.0,21.0,Egypt,2.0,1.0,2.0,Saudi Arabia,18.0,-1.0,-1.0,Russia,33.0,0.0,0.0,NaN
4,Porugal,B,6.0,0.0,0.0,2.0,3.0,Spain,3.0,-12.0,-31.0,Morocco,19.0,-1.0,-2.0,Iran,35.0,2.0,5.0,NaN


In [23]:
rankings.tail(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


In [24]:
print(matches.home_team.nunique(), rankings.country_full.nunique(), matches_2018.Team.nunique())
set(rankings.country_full) - set(matches.home_team) 

(241, 216, 32)


{'Bosnia and Herzegovina',
 'Brunei Darussalam',
 'Cape Verde Islands',
 'China PR',
 'Chinese Taipei',
 "C\xc3\xb4te d'Ivoire",
 'FYR Macedonia',
 'IR Iran',
 'Kyrgyz Republic',
 'Myanmar',
 'Netherlands Antilles',
 'RCS',
 'Republic of Ireland',
 'Serbia and Montenegro',
 'St Kitts and Nevis',
 'St Lucia',
 'St Vincent and the Grenadines',
 'S\xc3\xa3o Tom\xc3\xa9 e Pr\xc3\xadncipe',
 'Timor-Leste',
 'US Virgin Islands',
 'Zaire'}

In [25]:
set(matches_2018.Team) - set(matches.home_team) 

{nan, 'Columbia', 'Costarica', 'IRAN', 'Korea', 'Porugal'}

In [26]:
set(matches_2018.Team) - set(rankings.country_full) 

{nan, 'Columbia', 'Costarica', 'IRAN', 'Korea', 'Porugal'}

In [27]:
# clean up data
rankings = rankings.replace({'IR Iran': 'Iran', 
                             'China PR': 'China'})
matches_2018.dropna(how='all', inplace=True)
matches_2018 = matches_2018.replace({'Columbia': 'Colombia',
                                     'Costarica': 'Costa Rica',
                                     'IRAN': 'Iran',
                                     'Porugal': 'Portugal',
                                     'Korea': 'Korea Republic'})

In [28]:
print(set(matches_2018.Team) - set(matches.home_team) )
print(set(matches_2018.Team) - set(rankings.country_full))

set([])
set([])


## Feature engineering
What to predict: goal difference between two teams given teams' historical data
- Team level features:
    - ranking 
    - home/away
    - history with the opponent (TODO)

In [32]:
# ranking is monthly. join with matches on year-month
rankings['year_month'] = rankings.rank_date.apply(lambda x: x[:-3])
matches['year_month'] = matches.date.apply(lambda x: x[:-3])
X = pd.merge(rankings, matches, 
             left_on=['country_full', 'year_month'],
             right_on=['home_team', 'year_month'])

In [34]:
X = pd.merge(rankings, X,
             left_on=['country_full', 'year_month'],
             right_on=['away_team', 'year_month'],
             suffixes=('_away', '_home'))

In [55]:
# create features
for c in [u'rank_away', 
       u'total_points_away', u'previous_points_away', u'rank_change_away',
       u'cur_year_avg_away', u'cur_year_avg_weighted_away',
       u'last_year_avg_away', u'last_year_avg_weighted_away',
       u'two_year_ago_avg_away', u'two_year_ago_weighted_away',
       u'three_year_ago_avg_away', u'three_year_ago_weighted_away']:
    feat_name = c[:-5]
    X[feat_name + '_diff'] = X[feat_name + '_home'] - X[c]

### create outcome variable: 
- 1: home team won 
- 0: draw 
- 1: away team won

In [37]:
X['result'] = X.home_score - X.away_score
X.result = X.result.clip(upper=1, lower=-1)

In [38]:
X.result.value_counts()

 1    8634
-1    4898
 0    4311
Name: result, dtype: int64

## Model

In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

In [63]:
features = X.filter(like='diff').columns.tolist() + ['neutral']
features

[u'rank_diff',
 u'total_points_diff',
 u'previous_points_diff',
 u'rank_change_diff',
 u'cur_year_avg_diff',
 u'cur_year_avg_weighted_diff',
 u'last_year_avg_diff',
 u'last_year_avg_weighted_diff',
 u'two_year_ago_avg_diff',
 u'two_year_ago_weighted_diff',
 u'three_year_ago_avg_diff',
 u'three_year_ago_weighted_diff',
 'neutral']

In [79]:
X.columns

Index([                   u'rank_away',            u'country_full_away',
                  u'country_abrv_away',            u'total_points_away',
               u'previous_points_away',             u'rank_change_away',
                  u'cur_year_avg_away',   u'cur_year_avg_weighted_away',
                 u'last_year_avg_away',  u'last_year_avg_weighted_away',
              u'two_year_ago_avg_away',   u'two_year_ago_weighted_away',
            u'three_year_ago_avg_away', u'three_year_ago_weighted_away',
                 u'confederation_away',               u'rank_date_away',
                         u'year_month',                    u'rank_home',
                  u'country_full_home',            u'country_abrv_home',
                  u'total_points_home',         u'previous_points_home',
                   u'rank_change_home',            u'cur_year_avg_home',
         u'cur_year_avg_weighted_home',           u'last_year_avg_home',
        u'last_year_avg_weighted_home',        u'tw

In [97]:
X = X.set_index('date')
X.index = pd.to_datetime(X.index)

In [102]:
XX = X[X.tournament != 'Friendly']
XX = XX['1996':]
XX.shape, X.shape

((10329, 54), (17843, 54))

In [103]:
X_train, X_test, y_train, y_test = train_test_split(XX[features], XX.result)
X_train.shape, X_test.shape

((7746, 13), (2583, 13))

In [112]:
rf_model = RandomForestClassifier(n_estimators=200, max_depth=5)
rf_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=200, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [113]:
y_pred_prob = rf_model.predict_proba(X_test)
y_pred = rf_model.predict(X_test)

In [114]:
accuracy_score(y_test, y_pred)

0.64150212930700734

In [107]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)

[(0.37320070255829629, u'rank_diff'),
 (0.27524635070306852, u'previous_points_diff'),
 (0.10057189984882939, u'rank_change_diff'),
 (0.052940742422229195, u'total_points_diff'),
 (0.047445164735837313, u'cur_year_avg_diff'),
 (0.044856264728852853, u'cur_year_avg_weighted_diff'),
 (0.026723544646453764, u'three_year_ago_weighted_diff'),
 (0.017962162163462737, u'three_year_ago_avg_diff'),
 (0.017530804693062597, 'neutral'),
 (0.014096780678466632, u'two_year_ago_weighted_diff'),
 (0.012656988379221867, u'two_year_ago_avg_diff'),
 (0.0083921027347771942, u'last_year_avg_weighted_diff'),
 (0.008376491707441815, u'last_year_avg_diff')]

In [108]:
XX.describe()

,rank_away,total_points_away,previous_points_away,rank_change_away,cur_year_avg_away,cur_year_avg_weighted_away,last_year_avg_away,last_year_avg_weighted_away,two_year_ago_avg_away,two_year_ago_weighted_away,...,previous_points_diff,rank_change_diff,cur_year_avg_diff,cur_year_avg_weighted_diff,last_year_avg_diff,last_year_avg_weighted_diff,two_year_ago_avg_diff,two_year_ago_weighted_diff,three_year_ago_avg_diff,three_year_ago_weighted_diff
count,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,...,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000,10329.000000
mean,84.044438,161.414911,425.039791,0.936296,82.926981,82.926981,79.831503,39.915834,76.349205,22.904802,...,10.966599,0.438087,2.285995,2.285995,1.702848,0.851433,2.135063,0.640489,2.947743,0.589535
std,53.390238,299.966253,305.256554,7.677086,159.561105,159.561105,156.825664,78.412882,154.114082,46.234250,...,301.897149,10.778052,121.232711,121.232711,126.261995,63.130977,129.764498,38.929345,126.290510,25.258034
min,1.000000,0.000000,0.000000,-62.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1757.000000,-89.000000,-1056.290000,-1056.290000,-854.340000,-427.170000,-1106.460000,-331.940000,-963.140000,-192.630000
25%,39.000000,0.000000,180.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-127.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,79.000000,0.000000,409.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,123.000000,241.120000,597.000000,3.000000,107.770000,107.770000,100.000000,50.000000,86.550000,25.970000,...,154.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,209.000000,1770.060000,1887.000000,82.000000,1126.040000,1126.040000,1169.570000,584.790000,1158.660000,347.600000,...,1596.000000,78.000000,857.800000,857.800000,1058.910000,529.450000,1113.520000,334.060000,1062.640000,212.530000


## Simulations

### group rounds
- win - 3 points
- draw - 1 point
- lose - 0 point

In [116]:
matches_2018.head()

,Team,Group,Previous appearances,Previous titles,Previous finals,Previous semifinals,Current FIFA rank,First match against,Match index,history with first opponent W-L,history with first opponent goals,Second match against,Match index.1,history with second opponent W-L,history with second opponent goals,Third match against,Match index.2,history with third opponent W-L,history with third opponent goals,Unnamed: 19
0,Russia,A,10.0,0.0,0.0,1.0,65.0,Saudi Arabia,1.0,-1.0,-2.0,Egypt,17.0,NaN,NaN,Uruguay,33.0,0.0,0.0,NaN
1,Saudi Arabia,A,4.0,0.0,0.0,0.0,63.0,Russia,1.0,1.0,2.0,Uruguay,18.0,1.0,1.0,Egypt,34.0,-5.0,-5.0,NaN
2,Egypt,A,2.0,0.0,0.0,0.0,31.0,Uruguay,2.0,-1.0,-2.0,Russia,17.0,NaN,NaN,Saudi Arabia,34.0,5.0,5.0,NaN
3,Uruguay,A,12.0,2.0,2.0,5.0,21.0,Egypt,2.0,1.0,2.0,Saudi Arabia,18.0,-1.0,-1.0,Russia,33.0,0.0,0.0,NaN
4,Portugal,B,6.0,0.0,0.0,2.0,3.0,Spain,3.0,-12.0,-31.0,Morocco,19.0,-1.0,-2.0,Iran,35.0,2.0,5.0,NaN


In [168]:
matches_2018.set_index('Team', inplace=True)

In [153]:
# prepare features for the 2018 matches - get from latest ranking
def get_features(country1, country2):
    feats = [c[:-5] for c in features[:-1]]
    country1_feats = rankings[rankings.country_full == country1].iloc[-1, :][feats]
    country2_feats = rankings[rankings.country_full == country2].iloc[-1, :][feats]
#     print(country1_feats, country2_feats)
    final_feats = country1_feats - country2_feats
    # add home information
    if country1 == 'Russia' or country2 == 'Russia':
        # TODO: need to change the sign accordingly - Russia is home. use home - away
        final_feats['neutral'] = False
    else:
        final_feats['neutral'] = True
    return final_feats

In [178]:
# get_features('Russia', 'Germany')#.values.reshape(1, -1).shape

(1, 13)

In [169]:
matches_2018.head()

,Group,Previous appearances,Previous titles,Previous finals,Previous semifinals,Current FIFA rank,First match against,Match index,history with first opponent W-L,history with first opponent goals,...,Match index.1,history with second opponent W-L,history with second opponent goals,Third match against,Match index.2,history with third opponent W-L,history with third opponent goals,Unnamed: 19,points,total_prob
Team,,,,,,,,,,,,,,,,,,,,,
Russia,A,10.0,0.0,0.0,1.0,65.0,Saudi Arabia,1.0,-1.0,-2.0,...,17.0,NaN,NaN,Uruguay,33.0,0.0,0.0,NaN,0,0
Saudi Arabia,A,4.0,0.0,0.0,0.0,63.0,Russia,1.0,1.0,2.0,...,18.0,1.0,1.0,Egypt,34.0,-5.0,-5.0,NaN,0,0
Egypt,A,2.0,0.0,0.0,0.0,31.0,Uruguay,2.0,-1.0,-2.0,...,17.0,NaN,NaN,Saudi Arabia,34.0,5.0,5.0,NaN,0,0
Uruguay,A,12.0,2.0,2.0,5.0,21.0,Egypt,2.0,1.0,2.0,...,18.0,-1.0,-1.0,Russia,33.0,0.0,0.0,NaN,0,0
Portugal,B,6.0,0.0,0.0,2.0,3.0,Spain,3.0,-12.0,-31.0,...,19.0,-1.0,-2.0,Iran,35.0,2.0,5.0,NaN,0,0


In [182]:
from itertools import combinations

opponents = ['First match \nagainst', 'Second match\n against', 'Third match\n against']

matches_2018['points'] = 0
matches_2018['total_prob'] = 0

for group in set(matches_2018['Group']):
    print('___Group {}:___'.format(group))
    for home, away in combinations(matches_2018[matches_2018.Group == group].index, 2):
        print("{} vs. {}: ".format(home, away))
        feats = get_features(home, away)
        pred_prob = rf_model.predict_proba(feats.values.reshape(1, -1))
        result = rf_model.predict(feats.values.reshape(1, -1))
        matches_2018.loc[home, 'total_prob'] += pred_prob[0, 2]
        matches_2018.loc[away, 'total_prob'] += pred_prob[0, 0]
#         print('Result: {}'.format(result))
        if result == 1: # home win
            matches_2018.loc[home, 'points'] += 3
            print("  {} wins ({})".format(home, pred_prob[0, 2]))
        elif result == 0: # draw
            matches_2018.loc[home, 'points'] += 1
            matches_2018.loc[away, 'points'] += 1
            print("  Draw ({})".format(home, pred_prob[0, 1]))
        else: # away win 
            matches_2018.loc[away, 'points'] += 3
            print("  {} wins ({})".format(away, pred_prob[0, 0]))
#         print("\n")

___Group A:___
Russia vs. Saudi Arabia: 
  Russia wins (0.514278967579)
Russia vs. Egypt: 
  Egypt wins (0.363366226911)
Russia vs. Uruguay: 
  Uruguay wins (0.72218639174)
Saudi Arabia vs. Egypt: 
  Egypt wins (0.467653550607)
Saudi Arabia vs. Uruguay: 
  Uruguay wins (0.721320772063)
Egypt vs. Uruguay: 
  Uruguay wins (0.655153106521)
___Group C:___
France vs. Australia: 
  France wins (0.627727898)
France vs. Peru: 
  France wins (0.554344981265)
France vs. Denmark: 
  France wins (0.483642785088)
Australia vs. Peru: 
  Peru wins (0.493018993963)
Australia vs. Denmark: 
  Denmark wins (0.511380335974)
Peru vs. Denmark: 
  Peru wins (0.405976330341)
___Group B:___
Portugal vs. Spain: 
  Portugal wins (0.641281538772)
Portugal vs. Morocco: 
  Portugal wins (0.648203739014)
Portugal vs. Iran: 
  Portugal wins (0.671704412291)
Spain vs. Morocco: 
  Spain wins (0.635370230493)
Spain vs. Iran: 
  Spain wins (0.631575412714)
Morocco vs. Iran: 
  Morocco wins (0.365414794685)
___Group E:___

In [191]:
next_round = matches_2018.groupby('Group').apply(lambda x: 
                                    x.sort_values(by=['points', 'total_prob'], ascending=False).iloc[:2, :])


next_round = next_round.drop('Group', axis=1).reset_index()[['Group', 'Team']]

In [206]:
pairing = [0,3,4,7,8,11,12,15,1,2,5,6,9,10,13,14]

next_round = next_round.loc[pairing]
next_round.set_index('Team', inplace=True)

In [207]:
next_round

,Group
Team,
Uruguay,A
Spain,B
France,C
Iceland,D
Brazil,E
Mexico,F
Belgium,G
Colombia,H
Egypt,A


In [210]:
finals = ['round_of_16', 'quarterfinal', 'semifinal', 'final']

for f in finals:
    print("___Starting of the {}___".format(f))
    iterations = int(len(next_round) / 2)
    winners = []

    for i in range(iterations):
        home = next_round.index[i*2]
        away = next_round.index[i*2+1]
        print("{} vs. {}: ".format(home, away))
        feats = get_features(home, away)
        pred_prob = rf_model.predict_proba(feats.values.reshape(1, -1))
        result = rf_model.predict(feats.values.reshape(1, -1))
        
        if result == 1: # home win
            print("  {} wins ({})".format(home, pred_prob[0, 2]))
            winners.append(home)
        elif result == 0: # draw
            print("  Draw ({})".format(home, pred_prob[0, 1]))
            if pred_prob[0, 2] > pred_prob[0, 0]:
                winners.append(home)
            else:
                winners.append(away)
        else: # away win 
            print("  {} wins ({})".format(away, pred_prob[0, 0]))
            winners.append(away)
                
    next_round = next_round.loc[winners]
    print("\n")

___Starting of the round_of_16___
Uruguay vs. Spain: 
  Uruguay wins (0.386447980859)
France vs. Iceland: 
  France wins (0.624971695885)
Brazil vs. Mexico: 
  Brazil wins (0.598792508619)
Belgium vs. Colombia: 
  Belgium wins (0.619035020526)
Egypt vs. Portugal: 
  Portugal wins (0.664331367187)
Peru vs. Argentina: 
  Argentina wins (0.417104376248)
Switzerland vs. Germany: 
  Germany wins (0.574641806251)
England vs. Poland: 
  Poland wins (0.416143093596)


___Starting of the quarterfinal___
Uruguay vs. France: 
  France wins (0.411317494268)
Brazil vs. Belgium: 
  Brazil wins (0.41942602695)
Portugal vs. Argentina: 
  Portugal wins (0.490709179857)
Germany vs. Poland: 
  Germany wins (0.620537288115)


___Starting of the semifinal___
France vs. Brazil: 
  Brazil wins (0.452487329812)
Portugal vs. Germany: 
  Germany wins (0.47703693542)


___Starting of the final___
Brazil vs. Germany: 
  Germany wins (0.403897467014)


